# [<img src="logo.png" width="100" alt="drawing">](https://sites.google.com/view/cryspy/main) CrysPy 

**Web application** of the crystallographic python library CrysPy. 
Upload a ".rcif" file to get started with the CrysPy library, or use one of the examples that can also be found on [the CrysPy site](https://sites.google.com/view/cryspy/examples).

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy
import ipywidgets
from ipywidgets import Output, Button, widgets, Layout
from IPython.display import HTML, clear_output, display
from base64 import b64encode

In [19]:
import cryspy
from cryspy.B_parent_classes.cl_4_global import get_table_html_for_variables
from cryspy.H_functions_global.powder_experiments import report_powder_experiments

In [15]:
#Reference: https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

import requests
import io
def download_file_from_google_drive(id):
    destination = io.BytesIO()
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)
    s_content = destination.getvalue().decode("ASCII")
    return s_content 

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    for chunk in response.iter_content(CHUNK_SIZE):
        if chunk: # filter out keep-alive new chunks
            destination.write(chunk)

In [4]:
# global parameters
DDATA = {
    "rcif_obj": cryspy.str_to_globaln("global_"), 
    "file_rcif_name": "out.rcif"
}
L_EXAMPLE = [
    ("Single cryspy Ho2Ti2O7", "rhochi_single_susceptibility_Ho2Ti2O7.rcif", "1xG6Dz7f78XinH78uPNu0Mo6eU8w5Lh0L"),
    ("Powder 1D Ho2Ti2O7", "rhochi_polarized_powder_1d_Ho2Ti2O7_5K_1T.rcif", "18qdPs558CN39ABkR3JqtfNgvpn4SgCQo"),
    ("Powder 2D DyAl garnet", "rhochi_polarized_powder_2d_DyAl_5K_5T.rcif", "1GKnDsdKbYW6vhXuNQaS_jn1SddB6Viul"),
    ("Unpolarized neutron diffraction PbSo4", "rhochi_unpolarized_powder_1d_PbSO4.rcif", "1wax6x5wnoFiQXArTWbWv-E3YTWUKF6EV"),
    ("X-ray diffraction PbSo4", "rhochi_unpolarized_powder_1d_xrays_PbSO4.rcif", "1eQZLb613h7c0J5QA8fLlqhUCW6tI_-06"),
    ("TOF neutron diffraction", "rhochi_unpolarized_tof_powder_CeCuAl.rcif", "1qflJs3uXdG-O35rTGyyTXEqRa9WKELmX"),
    ("Spin density (MEM) YTiO3", "mempy_spin_density_YTiO3.rcif", "1S8YPVcNyYKEkP8j4RkUB7X9vn85bm435"),
    ("Magnetization density (MEM) Yb2Ti2O7", "mempy_magnetization_density_Yb2Ti2O7_2K_1T.rcif", "1PQmBK0xDu7Zxs-QzmFAzX0HtJnGMQkdY"),
    ]


In [5]:
# global widgets

W_OUT = widgets.Output(layout=Layout(justify_content='center'))

In [6]:
LAYOUT_50 = Layout(
    display='flex',    
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='49%',
)
LAYOUT_100 = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center',
    # border='solid 2px',
    align_items='stretch',
    width='98%',
    height="400px",
)

In [7]:
# page MENU
def draw_page_menu():
    widget_fo = widgets.FileUpload(
        description = "Upload RCIF",
        accept='.rcif',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False,  # True to accept multiple files upload else False
    )

    def change_output(ddata):
        s_cont = ddata["new"][0]["content"].tobytes().decode("ASCII")
        DDATA["rcif_obj"] = cryspy.str_to_globaln(s_cont)
        W_OUT.clear_output()
        with W_OUT:
            print(ddata["new"][0]["name"])
            DDATA["file_rcif_name"]=ddata["new"][0]["name"]
            print("File was uploaded")
        DISPLAY_NEW_OBJECT()
    
    widget_fo.observe(change_output, names="value")
    
    def trigger_download(text, filename, kind='text/json'):
        # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
        content_b64 = b64encode(text.encode()).decode()
        data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
        js_code = f"""
            var a = document.createElement('a');
            a.setAttribute('download', '{filename}');
            a.setAttribute('href', '{data_url}');
            a.click()
        """
        with W_OUT:
            clear_output()
            display(HTML(f'<script>{js_code}</script>'))


    
    btn_download = Button(description='Download it to PC')
    
    def download_rcif_file(e=None):
        file_name = DDATA["file_rcif_name"]
        if DDATA["rcif_obj"] is not None:
            trigger_download(DDATA["rcif_obj"].to_cif(), file_name, kind='text/plain')
        else:
            with W_OUT:
                print("File is not specified")
        
    btn_download.on_click(download_rcif_file)

    page_menu = widgets.Box(
        [widget_fo, btn_download],
        layout=Layout(justify_content="center"))
    return page_menu

In [8]:
# page object
def draw_page_object():
    
    select_data_item = widgets.Select(
        options=[],
        description='',
        rows=10,
        disabled=False,
        layout=LAYOUT_50,
        )
    select_loop_item = widgets.Select(
        options=[],
        description='',
        disabled=True,
        rows=10,
        layout=LAYOUT_50,
        )

    cif_text = widgets.Textarea(
        value='',
        placeholder="""Put the cif file and press "..add to data blocks" button to upload information about crystal to CrysPy.""",
        description='',
        disabled=False, # True
        layout=LAYOUT_100,
    )
    
    button_save_changes = widgets.Button(
        description="Save changes of current item or ...", 
        layout=Layout(justify_content="center", width="98%"))

    button_add_to_datas = widgets.Button(
        description=".. add to data blocks", 
        layout=Layout(justify_content="center", width="49%"))
    
    button_add_to_items = widgets.Button(
        description=".. add to items", 
        layout=Layout(justify_content="center", width="49%"))

    button_del_data = widgets.Button(
        description="Delete choosen data block", 
        layout=Layout(justify_content="center", width="49%"))
    
    button_del_item = widgets.Button(
        description="Delete choosen item", 
        layout=Layout(justify_content="center", width="49%"))

    def func_loop(dinfo_data_item):
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        W_OUT.clear_output()
        with W_OUT:
            plt.close()
            l_plots = data_item.plots()
            if len(l_plots)>0:
                plt.show()            
            display(HTML(data_item.report_html()))
            # display(HTML(data_item._repr_html_()))
        if isinstance(data_item, cryspy.DataN):
            l_loop_item_name = [item.get_name() for item in data_item]
            select_loop_item.disabled = False
            select_loop_item.options = l_loop_item_name
            if len(l_loop_item_name) > 0:
                loop_item = data_item[select_loop_item.index]
                if loop_item is not None:
                    cif_text.value = loop_item.to_cif()
                    cif_text.disabled = False
                else:
                    cif_text.value = ""
                    cif_text.disabled = False # True
            else:
                cif_text.value = ""
                cif_text.disabled = False # True
        else:
            cif_text.value = data_item.to_cif()
            select_loop_item.options = ()
            select_loop_item.disabled = True
            cif_text.disabled = False
        return
    
    select_data_item.observe(func_loop, names="index")
    
    def show_cif(dinfo_loop_item):
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        loop_item = l_data_item[loop_index]
        cif_text.value = loop_item.to_cif()
        cif_text.disabled = False
        loop_item_report = loop_item.report()
        
        W_OUT.clear_output()
        with W_OUT:
            plt.close()
            l_loop_item_plots = loop_item.plots()
            if len(l_loop_item_plots)>0:
                plt.show()
            display(HTML(loop_item.report_html()))
            # display(HTML(loop_item._repr_html_()))

        
    select_loop_item.observe(show_cif, names="index")
    
    def save_cif(b):        
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            loop_item_new = globaln_new.items[0]
            if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                l_data_item.items.pop(loop_index)
                l_data_item.items.insert(loop_index, loop_item_new)

    def add_to_globaln(b):        
        data_index = select_data_item.index
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            l_data_loop_item_new = globaln_new.items
            for data_loop_item_new in l_data_loop_item_new:
                if isinstance(data_loop_item_new, (cryspy.LoopN, cryspy.ItemN, cryspy.DataN)):
                    DDATA["rcif_obj"].items.append(data_loop_item_new)
            DISPLAY_NEW_OBJECT()

    def add_to_datan(b):        
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            l_loop_item_new = globaln_new.items
            for loop_item_new in l_loop_item_new:
                if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                    l_data_item.items.append(loop_item_new)
            DISPLAY_NEW_OBJECT()
    
    def del_data(b):        
        data_index = select_data_item.index
        if data_index is not None:
            DDATA["rcif_obj"].items.pop(data_index)
            DISPLAY_NEW_OBJECT()

    def del_item(b):        
        data_index = select_data_item.index
        if data_index is not None:
            l_data_item = DDATA["rcif_obj"].items[data_index]
            loop_index = select_loop_item.index
            if loop_index is not None:
                l_data_item.items.pop(loop_index)
                DISPLAY_NEW_OBJECT()


    button_save_changes.on_click(save_cif)
    button_add_to_datas.on_click(add_to_globaln)
    button_add_to_items.on_click(add_to_datan)
    button_del_data.on_click(del_data)
    button_del_item.on_click(del_item)
    
    items = [
        select_data_item, select_loop_item, 
        button_del_data, button_del_item,
        cif_text, 
        button_save_changes,button_add_to_datas,button_add_to_items]
    page_object = widgets.HBox(items, layout=Layout(flex_flow="row wrap", justify_content='center'))
    return page_object, select_data_item, select_loop_item, cif_text

In [9]:
def draw_page_cryspy_methods():

    def print_object_info(b):
        rcif_object = DDATA["rcif_obj"]
        
        ls_html = []
        ls_html.append(get_table_html_for_variables(rcif_object))
        ls_html.append(rcif_object.report_html())
        s_powder = report_powder_experiments(DDATA["rcif_obj"])
        W_OUT.clear_output()
        with W_OUT:
            display(HTML("\n".join(ls_html)))
            print(s_powder)
        return 

    def run_fitting(b):
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.rhochi_rietveld_refinement(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()            
            DISPLAY_NEW_OBJECT()

    def run_chi_sq(b):
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.rhochi_no_refinement(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()

    def run_mempy_sd(b):
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.mempy_spin_density_reconstruction(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()

    def run_mempy_md(b):
        W_OUT.clear_output()
        with W_OUT:
            print("Calculations are running...")
            cryspy.mempy_magnetization_density_reconstruction(DDATA["rcif_obj"])
            print("Calculations completed.")
            plt.close()
            l_plots = DDATA["rcif_obj"].plots()
            if len(l_plots)>0:
                plt.show()
            DISPLAY_NEW_OBJECT()

    button_info = widgets.Button(description="Info")
    button_calculations = widgets.Button(description="No fitting")
    button_fitting = widgets.Button(description="Fitting")
    button_mempy_sd = widgets.Button(description="Spin density (MEM)")
    button_mempy_md = widgets.Button(description="Magn. density (MEM)")

    items = [button_info, button_calculations, button_fitting, button_mempy_sd, button_mempy_md]
    
    button_info.on_click(print_object_info)
    button_fitting.on_click(run_fitting)
    button_calculations.on_click(run_chi_sq)
    button_mempy_sd.on_click(run_mempy_sd)
    button_mempy_md.on_click(run_mempy_md)

    page_cryspy_methods =  widgets.Box(
        items,
        layout=Layout(flex_flow="row wrap", justify_content='center'))
    return page_cryspy_methods

SyntaxError: invalid syntax (1988475966.py, line 13)

In [ ]:
def draw_page_examples():
    btn_1 = Button(description=L_EXAMPLE[0][0])
    btn_2 = Button(description=L_EXAMPLE[1][0])
    btn_3 = Button(description=L_EXAMPLE[2][0])
    btn_4 = Button(description=L_EXAMPLE[3][0])
    btn_5 = Button(description=L_EXAMPLE[4][0])
    btn_6 = Button(description=L_EXAMPLE[5][0])
    btn_7 = Button(description=L_EXAMPLE[6][0])
    btn_8 = Button(description=L_EXAMPLE[7][0])

    def download_example(b):
        l_name = [hh[0] for hh in L_EXAMPLE]
        ind_example = l_name.index(b.description)
        file_id = L_EXAMPLE[ind_example][2]
        s_content = download_file_from_google_drive(file_id)
        
        DDATA["rcif_obj"]=cryspy.str_to_globaln(s_content)
        DDATA["file_rcif_name"] = L_EXAMPLE[ind_example][1]
        DISPLAY_NEW_OBJECT()
        return 

    btn_1.on_click(download_example)
    btn_2.on_click(download_example)
    btn_3.on_click(download_example)
    btn_4.on_click(download_example)
    btn_5.on_click(download_example)
    btn_6.on_click(download_example)
    btn_7.on_click(download_example)
    btn_8.on_click(download_example)

    w_examples = widgets.Box(
            [btn_1, btn_2, btn_3, btn_4, btn_5, btn_6, btn_7, btn_8],
            layout=Layout(
                    flex_flow="row wrap", 
                    justify_content="center"))
    accordion = widgets.Accordion(
            children=[w_examples, ],
            titles=('.. or use one of the examples.',))
    page_examples = widgets.Box(
        [accordion, ],
        layout=Layout(justify_content="center"))
    return page_examples

In [13]:
page_menu = draw_page_menu()
page_examples = draw_page_examples()
page_object, SELECT_DATA_ITEM, SELECT_LOOP_ITEM, CIF_TEXT = draw_page_object()
page_cryspy_methods = draw_page_cryspy_methods()


In [ ]:
display(page_menu, page_examples, page_object)

In [32]:
display(HTML("<h2>Methods</h2>"))

In [ ]:
display(page_cryspy_methods, W_OUT)

In [12]:
def DISPLAY_NEW_OBJECT():
    l_data_item_name = [item.get_name() for item in DDATA["rcif_obj"].items]
    
    SELECT_DATA_ITEM.options = l_data_item_name
    # if (SELECT_DATA_ITEM.index is None) and len(l_data_item_name)>0:
    #     SELECT_DATA_ITEM.index = 0
    if (SELECT_DATA_ITEM.index is None):
        return
    
    data_index = SELECT_DATA_ITEM.index
    if data_index >= len(l_data_item_name):
        data_index = 0
    data_item = DDATA["rcif_obj"].items[data_index]
    if isinstance(data_item, cryspy.DataN):
        l_loop_item_name = [item.get_name() for item in data_item.items]
        SELECT_LOOP_ITEM.options = l_loop_item_name

        # if (SELECT_LOOP_ITEM.index is None) and len(l_loop_item_name)>0:
        #     SELECT_LOOP_ITEM.index = 0
        if (SELECT_LOOP_ITEM.index is None):
            return
        loop_index = SELECT_LOOP_ITEM.index
        if loop_index >= len(l_loop_item_name):
            loop_index = 0
        loop_item = data_item.items[0]
        CIF_TEXT.value = loop_item.to_cif()
    else:
        CIF_TEXT.value = data_item.to_cif()
        

()

In [ ]:
# select = widgets.Select(
#     options=['Linux', 'Windows', 'macOS'],
#     value='macOS',
#     # rows=10,
#     description='OS:',
#     disabled=False
# )
# download_output = Output()
# def change_output(value: str):
#     download_output.clear_output()
#     with download_output:
#         print(value["new"])
# select.observe(change_output, names="value")
# display(select, download_output)

In [ ]:
# widgets.Dropdown(
#     options=['1', '2', '3'],
#     value='2',
#     description='Number:',
#     disabled=False,
# )

In [ ]:
# widgets.Combobox(
#     # value='John',
#     placeholder='Choose Someone',
#     options=['Paul', 'John', 'George', 'Ringo'],
#     description='Combobox:',
#     ensure_option=True,
#     disabled=False
# )

In [ ]:
# from IPython.display import FileLink
# filename = "fff.rcif"
# FileLink(filename)